ライブラリの読み込み

In [1]:
from abc import ABC, abstractmethod
import json
import openai
from openai import OpenAI
from dotenv import load_dotenv, dotenv_values
import numpy as np
import os
#from embedder import OpenAIEmbedder
#from searcher import CosineNearestNeighborsFinder
#from chatBot import GPTBasedChatBot

# 環境変数を取得（OpenAI Key) 
env_name = "example.env" # following example.env template change to your own .env file name
config = dotenv_values(os.path.join(os.getcwd(), env_name), encoding='UTF-16LE')
api_key = config['openai_api_key']
#OpenAI.api_key = api_key
client = OpenAI(api_key=api_key)

RAG用データセットの読み込み

In [2]:
# ベクトル化するデータセット
texts = [
    "佐藤一郎は、東京生まれの35歳のプログラマーです。趣味は写真撮影とハイキング。新しい技術を学ぶことに情熱を注いでいます。",
    "鈴木花子は、北海道出身の28歳のイラストレーターです。猫を二匹飼っており、自然と動物を愛する心優しい人物です。",
    "田中健二は、大阪で小さなカフェを経営する45歳の起業家です。コーヒーに対する深い知識と情熱を持ち、地域社会に貢献しています。",
    "山本美咲は、福岡県出身の22歳の大学生です。法律を専攻しており、将来は人権に関わる仕事に就きたいと考えています。",
    "伊藤高志は、長野県の山の中で育った30歳の写真家です。自然の美しさを捉えることに特化し、国内外で展示会を開催しています。",
    "小林由紀子は、沖縄県出身の40歳の小学校教師です。子どもたちに芸術と文化の大切さを教えることに生きがいを感じています。",
]

RAG用データセットのベクトル化

In [10]:
# Embedderインターフェースの実装
def embed(texts: list[str]) -> list[list[float]]:
    # openai 1.10.0 で動作確認
    response = client.embeddings.create(input=texts, model="text-embedding-3-small")
    # レスポンスからベクトルを抽出
    return [data.embedding for data in response.data]

def save(texts: list[str], filename: str) -> bool:
    vectors = embed(texts)
    data_to_save = [
        {"id": idx, "text": text, "vector": vector}
        for idx, (text, vector) in enumerate(zip(texts, vectors))
    ]
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data_to_save, f, ensure_ascii=False, indent=4)
    print(f"{filename} に保存されました。")
    return True

DB構築（jsonファイル形式）

In [11]:
# データベクトル化とjsonファイル格納
save(texts, "sample_data.json")

sample_data.json に保存されました。


True

---------------------

ベクトル類似度検索

In [34]:
def _cosine_similarity(vec1: list[float], vec2: list[float]) -> float:
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    # openAI embeddingのベクトルを対象にする場合は正規化されているため、np.dot(vec1, vec2) だけでも良い
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

def find_nearest(data, vector: list[float], topk: int = 1) -> list[dict]:
    similarities = [
        (idx, _cosine_similarity(vector, item["vector"]))
        for idx, item in enumerate(data)
    ]
    # 類似度が高い順にソート
    sorted_similarities = sorted(similarities, key=lambda x: x[1], reverse=True)
    # Top-Kの結果を返す
    return [data[idx] for idx, _ in sorted_similarities[:topk]]

AIチャットボット回答の作成（プロンプト+RAG検索結果）

In [37]:
def generate_response(user_query: str, refs: list[str]) -> str:

    # GPTによる応答を生成
    context = "\n".join(refs) + "\n" #文字列を連結。連結毎に改行。

    prompt = f"以下の情報に基づいてユーザーの質問に答えてください:\n\n{context}\n\n質問: {user_query}\n答え:"
    #print("#" * 30)
    #print("#" * 30)
    print(f"\nprompt:\n {prompt}\n") # f"{<変数>}"
    #print("#" * 30)
    #print("#" * 30)

    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "user",
                "content": prompt,
            },
        ],
    )
    return completion.choices[0].message.content

JSONファイルの読み込み

In [15]:
def _load_data(data_file: str) -> list[dict]:
    with open(data_file, "r", encoding="utf-8") as f:
        return json.load(f)

AIチャットボットに対するメイン処理

In [38]:
data = _load_data("sample_data.json")
user_query: str = "アートを教えられる先生を探しています" # <変数>:<データ型>で、変数を宣言可能
user_query_vector: list[float] = embed([user_query])[0]

search_results: list[dict] = find_nearest(data, user_query_vector, topk=2)

response: str = generate_response(
    user_query, [search_result["text"] for search_result in search_results]
)

#print("*" * 30)
#print("*" * 30)
print("【AIの返答】")
print(response)

refs:['小林由紀子は、沖縄県出身の40歳の小学校教師です。子どもたちに芸術と文化の大切さを教えることに生きがいを感じています。', '伊藤高志は、長野県の山の中で育った30歳の写真家です。自然の美しさを捉えることに特化し、国内外で展示会を開催しています。']
context:小林由紀子は、沖縄県出身の40歳の小学校教師です。子どもたちに芸術と文化の大切さを教えることに生きがいを感じています。
伊藤高志は、長野県の山の中で育った30歳の写真家です。自然の美しさを捉えることに特化し、国内外で展示会を開催しています。


prompt:
 以下の情報に基づいてユーザーの質問に答えてください:

小林由紀子は、沖縄県出身の40歳の小学校教師です。子どもたちに芸術と文化の大切さを教えることに生きがいを感じています。
伊藤高志は、長野県の山の中で育った30歳の写真家です。自然の美しさを捉えることに特化し、国内外で展示会を開催しています。


質問: アートを教えられる先生を探しています
答え:

【AIの返答】
小林由紀子先生は芸術と文化の教育に生きがいを感じている小学校教師です。彼女がアートを教えることに熱心であるため、彼女が適格な候補の一人となります。


-------------

In [40]:
# リストの例
refs = ["apple", "banana", "cherry"]

# 要素を改行で連結して context 変数に代入
context = "\n".join(refs)

# context の中身を表示
print(context)

apple
banana
cherry


In [46]:
# 仮のpromptの値
prompt = "ユーザーの入力を受け付けてください"

# f文字列を使ってpromptの値を表示
print(f"\nprompt:\n\n {prompt}")


prompt:

 ユーザーの入力を受け付けてください


In [47]:
def generate_response(user_query: str, refs: list[str]) -> str:
    # ここに処理を実装する（例えば、ユーザークエリと参照データを組み合わせて適切な応答を生成する）
    response = f"ユーザーのクエリ: {user_query}, 参照データ: {refs}"
    return response

# 関数を呼び出す
user_query = "これはテストです"
refs = ["参照1", "参照2"]
result = generate_response(user_query, refs)
print(result)

ユーザーのクエリ: これはテストです, 参照データ: ['参照1', '参照2']
